# Advanced Python - Building Scalable Applications

### Module 7

#### Network socket programming
 -  Using ```asyncio``` and ```trio``` for concurrent socket programming

#### Web (RESTful) API using Python
 - An overview on RESTful API architecture
 - Consuming web API using python requests and httpx
 - An overview on WSGI and ASGI
 - WSGI / ASGI servers: ```gunicorn``` and ```uvicorn```
 - An overview on ```starlette``` ASGI framework
 - Creating and exposing web API using ```FastAPI``` framework
 - Discovering web APIs using ```/docs``` (Open-API) and ```/redocs``` interface
 - Creating validating models using ```pydantic```
 - Security Considerations: Authentication and Authorization techniques


In [4]:
class User:
    def greet(self):
        print("greetings...")

u = User()
u.greet()

hasattr(u, "greet")
g = getattr(u, "greet")
g()

greetings...
greetings...


In [5]:
# Handling binary data over text I/O streams

outs = open("/tmp/test.dat", "w")
outs

<_io.TextIOWrapper name='/tmp/test.dat' mode='w' encoding='UTF-8'>

In [15]:
import random
import base64 as encoder

nums = random.sample(range(255), 10)

data = bytes(nums)
data

#s = str(data, encoding="utf8")  # This fails!

s = encoder.b64encode(data).decode("utf8")

outs.write(s)
outs.close()

In [17]:
import base64
with open("/tmp/test.dat") as infile:
    content = infile.read()

data = base64.b64decode(content)
print(data)
print(list(data))

b'\xa4\xb1\xf6A\x05\x06a\x80\xb2\xae'
[164, 177, 246, 65, 5, 6, 97, 128, 178, 174]


#### RESTful architecture (overview)

Example:
    endpoint: /photos
    POST /photos?action=add  -> Create a new resource
    GET  /photos?action=list -> Retrieve all photos
    POST /photos?action=update&photo_id=112  -> Update a photo 
    GET  /photos?action=delete&phto_id=112 -> Delete a photo

The above semantics does not follow a "pure" RESTful architecture pattern - as they embed the actions within a query string

In RESTful architecture - we must focus on "resources".

Resource end-points are of two types:
   1. Collection of resources (resource collection endpoints)
      - Supported HTTP verbs: GET, POST
   2. A singular resource

Example:
 - Resource collection end-points
   /photos   # A plural / collective nouns indicate a resource collection
             # endpoint
   /album    # Another example of a resource collection endpoint

   GET /photos # Retrieve all photos as a list
   GET /photos?start=10&limit=5  # Retrieve a subset of photos as a list
   - The expected return value would always be a list

   POST /photos   # Create / Insert / Add a new resource into the
                  # collection

 - Singular resource
   /photos/123   # Always ends with a unique id to represent the resource

   GET /photos/123 # Retrieve a specific resource based on the id
   DELETE /photos/123 # Delete a specific resource based on the id
   PUT /photos/123    # Replace a resource for the given id
   PATCH /photos/123  # Update a resource for the given id





In [32]:
# OpenWeatherMap GeoCoding API example

API_KEY = "932c152d6ff8d185bfdd9d2a5f8e33e4"

owm_gc_ep = "http://api.openweathermap.org/geo/1.0/direct"
owm_gc_params = {
    "q": "Chennai", 
    "limit": "1", 
    "appid": API_KEY
}

#owm_wm_ep = "https://api.openweathermap.org/data/3.0/onecall"
owm_wm_ep = "https://api.openweathermap.org/data/2.5/weather"
owm_wm_params = {
    "appid": API_KEY,
    "units": "metric",
}

import requests

res = requests.get(owm_gc_ep, params=owm_gc_params)
if res.ok and "application/json" in res.headers["Content-Type"]:
    data = res.json()
    #print(data)
    lat, lon = data[0]["lat"], data[0]["lon"]
    print(f"Getting weather for {lat=}, {lon=}")

    owm_wm_params["lat"] = lat
    owm_wm_params["lon"] = lon

res = requests.get(owm_wm_ep, params=owm_wm_params)
if res.ok and "application/json" in res.headers["Content-Type"]:
    data = res.json()
    #print(data)
    print(data["main"]["temp"])


Getting weather for lat=13.0836939, lon=80.270186
34.06


In [43]:
# OpenWeatherMap GeoCoding API example using httpx

API_KEY = "932c152d6ff8d185bfdd9d2a5f8e33e4"

owm_gc_ep = "http://api.openweathermap.org/geo/1.0/direct"
owm_gc_params = {
    "q": "Chennai", 
    "limit": "1", 
    "appid": API_KEY
}

#owm_wm_ep = "https://api.openweathermap.org/data/3.0/onecall"
owm_wm_ep = "https://api.openweathermap.org/data/2.5/weather"
owm_wm_params = {
    "appid": API_KEY,
    "units": "metric",
}

import httpx

res = httpx.get(owm_gc_ep, params=owm_gc_params)
if res.is_success and "application/json" in res.headers["Content-Type"]:
    data = res.json()
    #print(data)
    lat, lon = data[0]["lat"], data[0]["lon"]
    print(f"Getting weather for {lat=}, {lon=}")

    owm_wm_params["lat"] = lat
    owm_wm_params["lon"] = lon

res = httpx.get(owm_wm_ep, params=owm_wm_params)
if res.is_success and "application/json" in res.headers["Content-Type"]:
    data = res.json()
    #print(data)
    print(data["main"]["temp"])


Getting weather for lat=13.0836939, lon=80.270186
31.27


In [44]:
import httpx

res = httpx.get("https://www.python.org/")
res

<Response [200 OK]>

In [45]:
res.is_success

True

In [58]:
import lxml.html as et
# import lxml.etree as et # for XML documents 


root = et.fromstring(res.text)
print(len(root))
print(root[0], root[1], root[2])
print(root[2][0])

body = root[2]
print(body.tag, body.text, body.attrib)

3
<!--<![endif]--> <Element head at 0x129ea19f0> <Element body at 0x129ea1040>
<Element div at 0x129ea1040>
body 

     {'class': 'python home', 'id': 'homepage'}


In [63]:
root.xpath("./body/div")

[<Element div at 0x129ddc780>]

In [66]:
root.xpath(".//div[@class='header-banner ']")

[<Element div at 0x129ddf5c0>]

In [69]:
root.xpath(".//a[@href]/@href")

['#content',
 '#python-network',
 '/',
 'https://www.python.org/psf/',
 'https://docs.python.org',
 'https://pypi.org/',
 '/jobs/',
 '/community/',
 '#top',
 '/',
 'https://psfmember.org/civicrm/contribute/transact?reset=1&id=2',
 '#site-map',
 '#',
 'javascript:;',
 'javascript:;',
 'javascript:;',
 '#',
 'https://www.linkedin.com/company/python-software-foundation/',
 'https://fosstodon.org/@ThePSF',
 '/community/irc/',
 'https://twitter.com/ThePSF',
 '/about/',
 '/about/apps/',
 '/about/quotes/',
 '/about/gettingstarted/',
 '/about/help/',
 'http://brochure.getpython.info/',
 '/downloads/',
 '/downloads/',
 '/downloads/source/',
 '/downloads/windows/',
 '/downloads/macos/',
 '/download/other/',
 'https://docs.python.org/3/license.html',
 '/download/alternatives',
 '/doc/',
 '/doc/',
 '/doc/av',
 'https://wiki.python.org/moin/BeginnersGuide',
 'https://devguide.python.org/',
 'https://docs.python.org/faq/',
 'http://wiki.python.org/moin/Languages',
 'https://peps.python.org',
 'https

In [30]:
res.content

b'{"cod":401, "message": "Please note that using One Call 3.0 requires a separate subscription to the One Call by Call plan. Learn more here https://openweathermap.org/price. If you have a valid subscription to the One Call by Call plan, but still receive this error, then please see https://openweathermap.org/faq#error401 for more info."}'

In [22]:
res.status_code

200

In [23]:
res.headers

{'Server': 'openresty', 'Date': 'Tue, 15 Apr 2025 05:54:24 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '543', 'Connection': 'keep-alive', 'X-Cache-Key': '/geo/1.0/direct?q=chennai', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Methods': 'GET, POST'}

In [24]:
res.json()

[{'name': 'Chennai',
  'local_names': {'fr': 'Chennai',
   'pl': 'Ćennaj',
   'de': 'Chennai',
   'ur': 'چنئی',
   'hi': 'चेन्नई',
   'ru': 'Ченнаи',
   'ml': 'ചെന്നൈ',
   'cs': 'Čennaí',
   'en': 'Chennai',
   'mr': 'चेन्नई',
   'ja': 'チェンナイ',
   'uk': 'Ченнаї',
   'bn': 'চেন্নাই',
   'ko': '첸나이',
   'he': "צ'נאי",
   'te': 'చెన్నై',
   'lt': 'Čenajus',
   'ar': 'تشيناي',
   'kn': 'ಚೆನ್ನೈ',
   'ta': 'சென்னை',
   'tr': 'Madras',
   'zh': '金奈'},
  'lat': 13.0836939,
  'lon': 80.270186,
  'country': 'IN',
  'state': 'Tamil Nadu'}]

In [71]:
rsess = requests.Session()
rsess

In [73]:
sess = httpx.Client()
sess.get("https://python.org/", follow_redirects=True)

<Response [200 OK]>

In [80]:
sess.get("https://expired.badssl.com/", verify=False)

TypeError: Client.get() got an unexpected keyword argument 'verify'

In [84]:
httpx.get("https://expired.badssl.com/", verify=False)

<Response [200 OK]>

In [79]:
httpx.get?

Signature:
httpx.get(
    url: 'URLTypes',
    *,
    params: 'QueryParamTypes | None' = None,
    headers: 'HeaderTypes | None' = None,
    cookies: 'CookieTypes | None' = None,
    auth: 'AuthTypes | None' = None,
    proxy: 'ProxyTypes | None' = None,
    proxies: 'ProxiesTypes | None' = None,
    follow_redirects: 'bool' = False,
    cert: 'CertTypes | None' = None,
    verify: 'VerifyTypes' = True,
    timeout: 'TimeoutTypes' = Timeout(timeout=5.0),
    trust_env: 'bool' = True,
) -> 'Response'
Docstring:
Sends a `GET` request.

**Parameters**: See `httpx.request`.

Note that the `data`, `files`, `json` and `content` parameters are not available
on this function, as `GET` requests should not include a request body.
File:      /opt/anaconda3/lib/python3.12/site-packages/httpx/_api.py
Type:      function

In [78]:
requests.get("https://expired.badssl.com/", verify=False)

/opt/anaconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'expired.badssl.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [85]:
requests.get?

Signature: requests.get(url, params=None, **kwargs)
Docstring:
Sends a GET request.

:param url: URL for the new :class:`Request` object.
:param params: (optional) Dictionary, list of tuples or bytes to send
    in the query string for the :class:`Request`.
:param \*\*kwargs: Optional arguments that ``request`` takes.
:return: :class:`Response <Response>` object
:rtype: requests.Response
File:      /opt/anaconda3/lib/python3.12/site-packages/requests/api.py
Type:      function

In [86]:
requests.Request?

Init signature:
requests.Request(
    method=None,
    url=None,
    headers=None,
    files=None,
    data=None,
    params=None,
    auth=None,
    cookies=None,
    hooks=None,
    json=None,
)
Docstring:     
A user-created :class:`Request <Request>` object.

Used to prepare a :class:`PreparedRequest <PreparedRequest>`, which is sent to the server.

:param method: HTTP method to use.
:param url: URL to send.
:param headers: dictionary of headers to send.
:param files: dictionary of {filename: fileobject} files to multipart upload.
:param data: the body to attach to the request. If a dictionary or
    list of tuples ``[(key, value)]`` is provided, form-encoding will
    take place.
:param json: json for the body to attach to the request (if files or data is not specified).
:param params: URL parameters to append to the URL. If a dictionary or
    list of tuples ``[(key, value)]`` is provided, form-encoding will
    take place.
:param auth: Auth handler or (user, pass) tuple.
:param 

In [89]:
from urllib.request import urlopen

res = urlopen("https://python.org/")
print(res.code)
print(res.read())

200
b'<!doctype html>\n<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->\n<!--[if IE 7]>      <html class="no-js ie7 lt-ie8 lt-ie9">          <![endif]-->\n<!--[if IE 8]>      <html class="no-js ie8 lt-ie9">                 <![endif]-->\n<!--[if gt IE 8]><!--><html class="no-js" lang="en" dir="ltr">  <!--<![endif]-->\n\n<head>\n    <!-- Google tag (gtag.js) -->\n    <script async src="https://www.googletagmanager.com/gtag/js?id=G-TF35YF9CVH"></script>\n    <script>\n      window.dataLayer = window.dataLayer || [];\n      function gtag(){dataLayer.push(arguments);}\n      gtag(\'js\', new Date());\n      gtag(\'config\', \'G-TF35YF9CVH\');\n    </script>\n    <!-- Plausible.io analytics -->\n    <script defer data-domain="python.org" src="https://plausible.io/js/script.js"></script>\n    <script defer data-domain="python.org" src="https://analytics.python.org/js/script.js"></script>\n\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" cont

#### Some popular old-school web API frameworks in Python:
  - Flask (sometimes with extensions like flask-restful / flask-restless)
  - Django + Piston
  - Pyramid
  - Web.py
  - aiohttpd
  - Falsy
  - Python hug
  - ...

#### "Pure" web API frameworks:
 - Falcon frameworks (and alternatives -> Python Responder, Vibora, Sanic, etc...)
 - FastAPI

#### Why FastAPI
 - Easy to learn and use
 - Feature-complete
 - High-performance
 - Very good model / resource abstraction with complete validation support using Pydantic
 - Built on-top of a extremely efficient and high-performance ASGI framework called starlette
 